In [1]:
filename = "Sample#1.pickle"

In [37]:
a.minDist = 40
a.getLable()

IndexError: arrays used as indices must be of integer (or boolean) type

In [2]:
import pickle
import numpy as np
import sys
sys.path.append("..")
import SHSlib  as sh
import cv2 as cv


class OfflineProcessing():
    
    def  __init__(self,filepath,floorCap=40,minDist=20):
        self.minDist = minDist
        self.filepath = filepath
        self.floorCap = floorCap
        
        # load pickelData
        with open(self.filepath, "rb") as f:
            self.dataList = pickle.load(f)
    
        self.ref = self.dataList[3]
        self.dataList = self.dataList[3:]
        
        #self.lable = self.getLable()

    def getRef():
        return  
    def getLable(self):
        return sh.analyse.getSeperation(self.ref,min_distance = self.minDist)
        
    def asd():
        ref_center = sh.analyse.getMomentum(ref_lables,ref_img, algorythm="C") 

            

a = OfflineProcessing("Sample#2.pickle")  




In [3]:
a

In [13]:
import pickle 

filename = "Sample#2.pickle"

with open(filename,'wb') as f:
    pickle.dump(a.dataList[1::3],f)